# Fetch and preprocess data

## Helper functions

In [77]:
# Define helper functions for the main script
import pandas as pd

# a function that takes a dictionary of lists of dataframes and sums up the row counts of all dataframes
def sum_row_count(dict):
    counts = []
    for key in dict.keys():
        for dflist in dict[key].values():
            counts.append(len(dflist))
    return sum(counts)

def assert_reconcilation(dataset, transaction_df, reconciliation_dict):
    if len(transaction_df) == sum_row_count(reconciliation_dict[dataset]['reconciliation']):
        print(f'All {dataset} transactions accounted for in reconciliation_dict')
    else:
        print(f'{len(transaction_df) - sum_row_count(reconciliation_dict[dataset]["reconciliation"])} {dataset} transactions not accounted for in reconciliation_dict')
    return len(transaction_df) - sum_row_count(reconciliation_dict[dataset]["reconciliation"])

def matchByDateAndValue(df1, df1cols, df2, df2cols, timeDelta='10 days', valueDelta=0.1, absolute=False):
    a = df1[df1cols]
    b = df2[df2cols]
    matched = []

    def naive_transaction_matcher(x):
        possible = b.drop(index=matched)
        possibleByDate = (
            possible[df2cols[0]] - x[df1cols[0]]).abs().sort_values(ascending=True)
        possibleByDate = possibleByDate[possibleByDate < pd.Timedelta(
            timeDelta)]
        if len(possibleByDate) == 0:
            return None
        possible = possible.loc[possibleByDate.index]
        possibleValues = possible[df2cols[1]].abs(
        ) if absolute else possible[df2cols[1]]
        targetValue = abs(x[df1cols[1]]) if absolute else x[df1cols[1]]
        possibleByAmount = (
            possibleValues - targetValue).abs().sort_values(ascending=True)
        possibleByAmount = possibleByAmount[possibleByAmount <
                                            possible.loc[possibleByAmount.index][df2cols[1]] * valueDelta]
        if len(possibleByAmount) == 0:
            return None
        id = int(possibleByAmount.index[0])
        matched.append(id)
        return id
    indexes = pd.DataFrame(a.apply(
        naive_transaction_matcher, axis=1).dropna().astype('int'), columns=["df2key"])
    return df1.join(indexes).join(df2, on="df2key")

from fuzzywuzzy import fuzz
def matchByDateAmountString(bank_transactions, platform_transactions, s1='bank.Detail', s2='oppositeAccount.name', min_similarity=80, max_days=2, max_diff=0.05):
    # Function to return a dataframe with bank transaction index as key and a list of tuples with platform transaction index and similarity score as value

    def is_date_within_range(bank_date, platform_date, max_days):
        return abs((bank_date - platform_date).days) <= max_days

    def is_amount_within_range(bank_transaction, platform_amount, max_diff):
        bank_amount = bank_transaction['bank.Debit'] if not (pd.isna(bank_transaction['bank.Debit']) or bank_transaction['bank.Debit'] == 0 ) else bank_transaction['bank.Credit']
        return abs(bank_amount - abs(platform_amount))/bank_amount <= max_diff if bank_amount != 0 else 0

    matches = {}
    for index, bank_row in bank_transactions.iterrows():
        match_list = []
        for platform_index, platform_row in platform_transactions.iterrows():

            if not is_date_within_range(bank_row['bank.Post Date'], platform_row['createdAt'], max_days):
                continue
            
            if not is_amount_within_range(bank_row, platform_row['amountInHostCurrency.value'], max_diff):
                continue

            # Calculate the similarity score
            similarity_score = fuzz.partial_ratio(bank_row[s1].lower(), platform_row[s2].lower())

            if similarity_score >= min_similarity:
                platform_id = platform_transactions.loc[platform_index]['id']
                match_list.append((platform_id, similarity_score))

        # Sort the matches by similarity score, in descending order
        match_list.sort(key=lambda x: x[1], reverse=True)

        # Ensure only matches with exactly one match are added to not add ambiguous matches
        if len(match_list) == 1:
            matches[bank_row['bank.accountindex']] = match_list

    # Convert the matches dictionary to a dataframe
    matches_df = pd.DataFrame([(k, v[0][0]) for k, v in matches.items() if len(v) > 0], columns=['bank_accountindex', 'platform_id'])

    return matches_df

## Fetch Data

Fetch data from Open Collective, Wise, Stripe and PayPal.

In [78]:
# Import libraries

import requests
import json
import time

# Set display options
pd.set_option('display.max_columns', None)

In [79]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

### Get data from Open Collective

Set the host on the platform to get data for, and the period.

In [80]:
# Host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [81]:
# if you want to reload the data from the API, set this to True, otherwise it will load from the cached file
reload_data = False

Get data from the Open Collective API.

In [82]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = f"https://opencollective.com/api/graphql/v2?personalToken={octoken}"

    # Your GraphQL query
    query = """
    query TransactionsTable(
      $hostAccount: AccountReferenceInput,
      $limit: Int!,
      $offset: Int!,
      $type: TransactionType,
      $paymentMethodType: [PaymentMethodType],
      $dateFrom: DateTime,
      $dateTo: DateTime,
      $kind: [TransactionKind],
      $includeIncognitoTransactions: Boolean,
      $includeGiftCardTransactions: Boolean,
      $includeChildrenTransactions: Boolean,
      $orderBy: ChronologicalOrderInput,
      $group: String,
      $includeHost: Boolean,
      $order: OrderReferenceInput
    ) {
      transactions(
        host: $hostAccount
        limit: $limit
        offset: $offset
        type: $type
        paymentMethodType: $paymentMethodType
        dateFrom: $dateFrom
        dateTo: $dateTo
        kind: $kind
        includeIncognitoTransactions: $includeIncognitoTransactions
        includeGiftCardTransactions: $includeGiftCardTransactions
        includeChildrenTransactions: $includeChildrenTransactions
        includeDebts: true
        orderBy: $orderBy
        group: $group
        includeHost: $includeHost
        order: $order
      ) {
        totalCount
        offset
        limit
        nodes {
          createdAt
          id
          merchantId
          group
          type
          kind
          description
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          amount {
            currency
            value
          }
          amountInHostCurrency {
            currency
            value
          }
          netAmount {
            currency
            value
          }
          netAmountInHostCurrency {
            currency
            value
          }
          paymentProcessorFee {
            currency
            value
          }
          balanceInHostCurrency {
            currency
            value
          }
          account {
            id
            legacyId
            name
            slug
            isIncognito
            type
            ... on AccountWithParent {
              parent {
                id
                name
                slug
                type
              }
            }
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
          }
          order {
            id
            description
            memo
            pendingContributionData {
              memo
            }
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
            }
            type
          }
          expense {
            id
            type
            description
            invoiceInfo
            tags
            virtualCard {
              id
            }
            payee {
              id
              name
              slug
              type
            }
          }
          payoutMethod {
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
          }
        }
      }
    }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:

        # Try catch around json_data = response.json() to avoid errors

        max_retries = 10  # Setting the maximum number of retries
        retries = 0  # Initial retry count
        response = ''

        while retries < max_retries:
            try:
                # Make the HTTP request
                response = requests.post(
                    url, json={'query': query, 'variables': variables}, headers=headers)
                json_data = response.json()
                break
            except Exception as e:
                print("Request failed. Attempt:", retries + 1)
                print(e)
                retries += 1
                time.sleep(10)

        if retries == max_retries:
            print("Maximum retry attempts reached. Exiting.")
        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break

        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 1 seconds to avoid server strain
        time.sleep(1)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
    if 'parent' in transaction['account']:
        transaction['account'] = transaction['account']['parent']
    return transaction

def post_process_transactions(all_transactions):
    return list(map(replace_account_with_parent, all_transactions))

if reload_data:
  all_transactions = fetch_transactions()
  # dump all transactions to a json file
  post_process_transactions(all_transactions)
  with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
    json.dump(all_transactions, f, indent=2)
  # convert the json file to a dataframe
  df_platform = pd.json_normalize(all_transactions)
  # save dataframe as pickle
  df_platform.to_pickle(
      f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

Get data from the Wise API.

In [83]:
# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })
            
        # Update the offset for the next iteration
        offset += limit

    return all_transfers

if reload_data:
    transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)
    # create transfers dataframe
    df_wise = pd.DataFrame()
    for transfer in transfers:
        # add to dataframe with concat
        df_wise = pd.concat([df_wise, pd.DataFrame([transfer])])
    # save dataframe as pickle
    df_wise.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

## Load Cached Data

Loading data from files with Stripe and Paypal transactions, and loading cached data from Wise and Paypal if available.

Data sources:
 * Platform data is retrieved from the Open Collective API
 * Wise data is retrieved from the Wise API
 * Stripe data is downloaded from https://dashboard.stripe.com/reports/balance Balance change from activity
   * "Balance change from activity" with Report format "Itemized" and Reporting category "All"
 * Paypal data is downloaded from https://www.paypal.com/reports/dlog
   * "Activity report" with Transaction type "Balance affecting" in CSV format
 * Bank data are itemized CSV exports
   * Columns names in export need to be: "Post Date", "Customer Ref", "Debit", "Credit", "BAI Description", "Description", "Detail", "Account Number"

In [84]:
# import datasets from local files

import glob

# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe_transactions.csv', low_memory=False)

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load paypal data
dir = f'data/{host}/{year}/paypal'
df_paypal = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_paypal = df_paypal.dropna(axis=1, how='all')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')


## Sanitizing Data

Explicitly keep only the columns we need. 

In [85]:
# Keep only the columns we need in Stripe

df_stripe = df_stripe[[
    'balance_transaction_id', 
    'created_utc', 
    'created', 
    'available_on_utc',
    'available_on',
    'currency',
    'gross',
    'fee',
    'net',
    'reporting_category',
    'source_id',
    'description',
    'customer_facing_amount',
    'customer_facing_currency',
    'automatic_payout_id',
    'automatic_payout_effective_at_utc',
    'automatic_payout_effective_at',
    'customer_id',
    'customer_description',
    'charge_id',
    'payment_intent_id',
    'charge_created_utc',
    'charge_created',
    'payment_method_type',
    'card_brand',
    'card_funding',
    'card_country',
    'payment_metadata[from]',
    'payment_metadata[to]'
]]


Strip trailing white spaces from column names.

In [86]:
# Strip trailing whitespace from column names

df_bank_1.columns = df_bank_1.columns.str.strip()
df_bank_2.columns = df_bank_1.columns.str.strip()
df_stripe.columns = df_stripe.columns.str.strip()
df_wise.columns = df_wise.columns.str.strip()
df_paypal.columns = df_paypal.columns.str.strip()
df_platform.columns = df_platform.columns.str.strip()

Convert created at columns to datetime.

In [87]:
# Convert date formats 

df_platform['createdAt'] = pd.to_datetime(df_platform['createdAt']).dt.tz_convert(None)

df_stripe['created_utc'] = pd.to_datetime(df_stripe['created_utc'], utc=True).dt.tz_convert(None)

df_wise['created'] = pd.to_datetime(df_wise['created'], utc=True).dt.tz_convert(None)

df_bank_1['Post Date'] = pd.to_datetime(df_bank_1['Post Date'], utc=True).dt.tz_convert(None)
df_bank_2['Post Date'] = pd.to_datetime(df_bank_2['Post Date'], utc=True).dt.tz_convert(None)

df_paypal['Datetime'] = pd.to_datetime(df_paypal['Date'] + ' ' + df_paypal['Time'], utc=True)
tz_offset = {
    'PST': -8,
    'PDT': -7,
}

import datetime
def fix_tz(row):
    tz=datetime.timezone(datetime.timedelta(hours=tz_offset[row['TimeZone']]))
    row['Datetime'] = pd.to_datetime(row['Datetime'].replace(tzinfo=tz)).tz_convert(None)
    return row
    
df_paypal = df_paypal.apply(fix_tz, axis=1)

Sanitize Paypal data.

In [88]:
# Sanitize Paypal data by removing commas from numbers and removing commas from Note column

df_paypal['Net'] = df_paypal['Net'].replace(',', '', regex=True).astype(dtype=float)
df_paypal['Gross'] = df_paypal['Gross'].replace(',', '', regex=True).astype(dtype=float)
df_paypal['Fee'] = df_paypal['Fee'].replace(',', '', regex=True).astype(dtype=float)
df_paypal['Balance'] = df_paypal['Balance'].replace(',', '', regex=True).astype(dtype=float)
df_paypal['Note'] = df_paypal[~df_paypal['Note'].isna()]['Note'].str.replace(',', '')
df_paypal['Type'] = df_paypal['Type'].str.strip()

## Filtering out Data

Filter out failed, cancelled and pending transactions from Stripe, Paypal and Wise as these have not affected the balance.

TODO: If we switch the Stripe datasource to only use the payouts file, this step is probably not necessary as that file only includes transactions that have affected the balance.

In [89]:
# only keep rows in paypal with status in ['Completed', 'Processed']
# df_paypal = df_paypal[df_paypal['Status'].isin(['Completed', 'Processed'])]
df_paypal = df_paypal[df_paypal['Balance Impact'] != 'Memo']

# remove cancelled from Wise
df_wise = df_wise[df_wise['status'] != 'cancelled']

## Combining

Combine the bank files into one dataframe. We can do this because the account number is stated in a column of the bank exports.

In [90]:
df_banks = pd.concat([df_bank_1, df_bank_2]).reset_index()

# create a column accountindex that concatenates bank.Account Number and bank.index to create a unique identifier for each bank transaction
df_banks['accountindex'] = df_banks['Account Number'].astype(str) + '-' + df_banks['index'].astype(str)

## Renaming

Rename columns to not confuse them after merging with other data sources later.

In [91]:
# Prefix wise columns with wise. and rename id to wise.id
df_wise = df_wise.set_index('id', drop=False).rename_axis(None).add_prefix('wise.')
# Prefix stripe columns with stripe. and rename id to stripe.id
df_stripe = df_stripe.set_index('balance_transaction_id', drop=False).rename_axis(None).add_prefix('stripe.')
# Prefix paypal columns with paypal. and rename id to paypal.id. Paypal Transaction ID is not a unique identifier, so we keep the index.
df_paypal = df_paypal.rename_axis(None).add_prefix('paypal.')
# Prefix bank columns with bank. and rename index to bank.id
df_banks = df_banks.rename_axis(None).add_prefix('bank.')

# Reconcile transactions

We are now ready to start matching transactions on Wise, Stripe, Paypal and bank accounts with known events on the platform.

Our goal is to be able to reconcile as many transactions as we can automatically. 

For accounting purposes the most important thing is to use the information on the platform to explain transactions on Wise, Stripe, Paypal and bank accounts. 

At the end of this section we will end up with the following outputs:

  * **Wise Reconciled**: Wise transactions matched with their corresponding platform transaction or a corresponding balance transfer from the bank.
  * **Wise Remainder**: Wise transactions for which no platform transaction or bank transaction was found.
  * **Stripe Reconciled**: Stripe transactions matched with their corresponding platform transaction or a corresponding balance transfer to the bank.
  * **Stripe Remainder**: Stripe transactions for which no platform transaction or bank transaction was found.
  * **Paypal Reconciled**: Paypal transactions matched with their corresponding platform transaction or a corresponding balance transfer to or from the bank.
  * **Paypal Remainder**: Paypal transactions for which no platform transaction or bank transaction was found.
  * **Bank Reconciled**: Bank transactions matched with corresponding platform transactions or balance transfers to/from Wise, Stripe or Paypal.
  * **Bank Remainder**: Bank transactions that have not been reconciled with the platform, Stripe, Wise or Paypal.
  * **Platform Reconciled**: Platform transactions that have been reconciled. 
  * **Platform Remainder**: Platform transactions for which no corresponding transaction has been found on Wise, Stripe, Paypal or bank accounts.

## Create reconciliation dict

We will save our reconciliation results in a dictionary.

In [92]:
reconciliation_dict = {
    'wise': {
        'all': df_wise,
        'reconciliation': {
            'reconciled_platform': {},
            'reconciled_bank': {},
            'unreconciled': {}
        }
    },
    'stripe': {
        'all': df_stripe,
        'reconciliation': {
            'reconciled_platform': {},
            'reconciled_bank': {},
            'reconciled_other': {},
            'unreconciled': {}
        },
    },
    'paypal': {
        'all': df_paypal,
        'reconciliation': {
            'reconciled_platform': {},
            'reconciled_bank': {},
            'reconciled_other': {},
            'unreconciled': {}
        },

    },
    'bank': {
        'all': df_banks,
        'reconciliation': {
            'reconciled_platform': {},
            'reconciled_wise': {},
            'reconciled_stripe': {},
            'reconciled_paypal': {},
            'reconciled_other': {},
            'unreconciled': {}
        },
    },
}

## Wise reconciliation

### Reconcile Wise with platform

Wise is pretty easy to reconcile since it should only be money out for expense and invoice payments and occational returns the funds could not be sent. We start by looking into the transactions.

In [93]:
df_wise.groupby(['wise.status', 'wise.sourceCurrency']).agg({'wise.sourceValue': ['count', 'sum']})

wise.sourceValue             
                                                     count          sum
wise.status           wise.sourceCurrency                              
funds_refunded        USD                               54    338773.20
outgoing_payment_sent EUR                                3      1719.00
                      GBP                                1        46.86
                      USD                             3405  12600866.08

Merge the Wise transactions table with the potentially Wise-related platform transactions table, joining on the merchantId on the platform with the Wise id from the Wise transaction table.

In [94]:
df_wise_platform = pd.merge(df_platform.dropna(subset=['merchantId']), df_wise.dropna(subset=['wise.id']).astype({'wise.id': 'string'}), left_on='merchantId', right_on='wise.id', how='inner')
df_wise_platform.drop_duplicates(subset=['wise.id'], inplace=True)

print(f'Successfully matched {len(df_wise_platform)} Wise transactions with platform transactions')

# remainder of Stripe transactions by filter out on the unique balance_transaction_id
df_left_wise = df_wise[~df_wise['wise.id'].astype({'wise.id': 'string'}).isin(df_wise_platform['wise.id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_wise)} out of {len(df_wise)}')

Successfully matched 3403 Wise transactions with platform transactions
Unreconciled Stripe transactions: 60 out of 3463


### Reconcile Wise with bank

Setting the known Wise trace account reference that appears on bank statements

In [95]:
WISE_TRACE_ACCOUNT = 'XXXXXX7206'

Finding the bank transactions that are known to have been transfers to the Wise account.

In [96]:
df_banks_wise = df_banks[df_banks['bank.Detail'].str.contains(WISE_TRACE_ACCOUNT)]

Finding the Wise transactions that correspond to the known Wise balance transfers from the bank accounts.

Because the bank transactions also cover transfer fees the amounts are not exact.

We use a function to find incoming balance transfers to the Wise account within 2 days and within 95% of the amount transferred from the bank.

TODO: When accounting we need to account for the balance transfer fees. 

In [97]:
# importing a function that can match rows in two dataframes by date and value

df_wise_bank = matchByDateAndValue(df_left_wise, ['wise.created', 'wise.sourceValue'], df_banks_wise, ['bank.Post Date', 'bank.Debit'] , '2 days', 0.05).dropna(subset=['df2key'])
df_left_wise = df_left_wise[~df_left_wise['wise.id'].isin(df_wise_bank['wise.id'])]
print(f"Wise transactions with a corresponding Bank transaction: {len(df_wise_bank)}")
print(f"Wise transactions without either a corresponding Platform transaction or a Bank transaction: {len(df_left_wise)} out of {len(df_wise)}")
print(f"Percentage of Wise transactions that have a corresponding transaction either on the Platform or on the Bank: {round((len(df_wise_bank)+len(df_wise_platform))/len(df_wise)*100, 2)}%")

Wise transactions with a corresponding Bank transaction: 34
Wise transactions without either a corresponding Platform transaction or a Bank transaction: 26 out of 3463
Percentage of Wise transactions that have a corresponding transaction either on the Platform or on the Bank: 99.25%


### Analyzing Wise remainder

In [98]:
df_left_wise.groupby(['wise.status', 'wise.sourceCurrency']).agg({'wise.sourceValue': ['count', 'sum']})

wise.sourceValue           
                                                     count        sum
wise.status           wise.sourceCurrency                            
funds_refunded        USD                               18  263010.86
outgoing_payment_sent USD                                8  361296.12

### Adding Wise to reconciliation dict

In [99]:
reconciliation_dict['wise']['reconciliation']['reconciled_bank']['wise_bank'] = df_wise_bank
reconciliation_dict['wise']['reconciliation']['reconciled_platform']['wise_platform'] = df_wise_platform
reconciliation_dict['wise']['reconciliation']['unreconciled']['unreconciled_wise'] = df_left_wise

In [100]:
assert_reconcilation('wise', df_wise, reconciliation_dict)

All wise transactions accounted for in reconciliation_dict


0

## Stripe reconciliation

OSC used Stripe in 2023 for incoming contributions and for virtual cards. That makes reconcilliation of Stripe slightly complicated. 

We start by analyzing the transactions.

In [101]:
df_stripe.groupby('stripe.reporting_category').agg({'stripe.gross': ['count', 'sum'], 'stripe.fee': ['sum'], 'stripe.net': ['sum']})

stripe.gross             stripe.fee  stripe.net
                                     count         sum        sum         sum
stripe.reporting_category                                                    
charge                              131783  4480808.61  425873.98  4054934.63
charge_failure                          18    -3618.00      43.05    -3661.05
dispute                                 47   -38726.33     705.00   -39431.33
dispute_reversal                        19      190.13    -270.00      460.13
fee                                    680    -3569.72       0.00    -3569.72
issuing_authorization_hold            2293  -251849.40       0.00  -251849.40
issuing_authorization_release         2294   251813.40       0.00   251813.40
issuing_dispute                          6      290.14       0.00      290.14
issuing_transaction                   2222  -213310.19       0.00  -213310.19
refund                                 508   -22306.58       0.00   -22306.58
topup                                    8   191000.00       0.00   191000.00
topup_reversal                           1    -5000.00       0.00    -5000.00

### Reconcile contributions

For each row in in the Stripe table, we find the corresponding row in df_platform, matching the Stripe id on the platform merchantId.

Note: It turns out that quite a large number of Stripe contributions do not get assigned STRIPE as paymentMethod.service on the platform. We need to merge on merchantId without first filtering out known Stripe transactions from the platform.

In [102]:
df_stripe_contributions = df_stripe[df_stripe['stripe.reporting_category'] == 'charge']
df_stripe_platform_contributions = pd.merge(df_platform.dropna(subset=['merchantId']), df_stripe_contributions.dropna(subset=['stripe.source_id']), left_on='merchantId', right_on='stripe.source_id', how='inner')
df_stripe_platform_contributions.drop_duplicates(subset=['stripe.balance_transaction_id'], inplace=True)

print(f'Successfully matched {len(df_stripe_platform_contributions)} Stripe transactions with platform contributions')

# remainder of Stripe transactions by filter out on the unique balance_transaction_id
df_left_stripe = df_stripe[~df_stripe['stripe.balance_transaction_id'].isin(df_stripe_platform_contributions['stripe.balance_transaction_id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')


Successfully matched 131332 Stripe transactions with platform contributions
Unreconciled Stripe transactions: 8547 out of 139879


Because paymentMethod is missing for a portion of Stripe contributions on the platform, we can't be sure about if there are platform transactions that have been created by a contribution that was supposed to go through Stripe but that never ended up on Stripe.

In [103]:
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Unreconciled Stripe transactions: 8547 out of 139879


### Reconcile refunds

In [104]:
# Getting the list of transactions that have been refunded in Stripe
df_stripe_refunds = df_left_stripe[df_left_stripe['stripe.reporting_category'] == 'refund']
# Get refunded fees from contributions from Stripe where stripe.reporting_category is fee and net is positive
df_stripe_refunded_fees = df_stripe[(df_stripe['stripe.reporting_category'] == 'fee') & (df_stripe['stripe.net'] > 0)]
# Concatenate the two dataframes
df_stripe_all_refunds = pd.concat([df_stripe_refunds, df_stripe_refunded_fees])

In [105]:
# Matching the refunds with the platform contributions on stripe.charge_id
df_stripe_platform_contrubution_refunds = pd.merge(df_platform.dropna(subset=['merchantId']), df_stripe_all_refunds.dropna(subset=['stripe.charge_id']), left_on='merchantId', right_on='stripe.charge_id', how='inner')
df_stripe_platform_contrubution_refunds.drop_duplicates(subset=['stripe.balance_transaction_id'], inplace=True)
print(f'Successfully matched {len(df_stripe_platform_contrubution_refunds)} Stripe transactions with refund transactions on platform')

Successfully matched 432 Stripe transactions with refund transactions on platform


In [106]:
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_platform_contrubution_refunds['stripe.balance_transaction_id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')


Unreconciled Stripe transactions: 8115 out of 139879


### Reconcile fees

In [107]:
df_stripe_application_fees = df_left_stripe[df_left_stripe['stripe.reporting_category'] == 'fee']
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_application_fees['stripe.balance_transaction_id'])]
print(f'Successfully identified {len(df_stripe_application_fees)} Stripe transactions as application fees')
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Successfully identified 467 Stripe transactions as application fees
Unreconciled Stripe transactions: 7648 out of 139879


### Reconcile Stripe virtual card transactions

#### Filter out hold and release transactions that balance out

Start by cancelling out issuing_authorization_hold with corresponding issuing_authorization_release as these do not affect the balance. 

Hold occurs when a virtual card is used to make a purchase, generating an authorization request. If approved, the amount is held from the Issuing balance until the authorization is captured, voided, or expires without capture. When an authorization is captured, the status is set to 'closed', and the held purchase amount is released.

We find the transactions that balance out and put them aside.

In [108]:
# get all virtual card transactions from Stripe
df_stripe_issuing = df_left_stripe[df_left_stripe['stripe.reporting_category'].isin(['issuing_authorization_hold', 'issuing_authorization_release', 'issuing_dispute', 'issuing_transaction'])]

In [109]:
# filter out all issuing_authorization_hold that have a corresponding issuing_authorization_release with the same stripe.source_id
df_stripe_issuing_authorization_hold = df_stripe_issuing[df_stripe_issuing['stripe.reporting_category'] == 'issuing_authorization_hold']
df_stripe_issuing_authorization_release = df_stripe_issuing[df_stripe_issuing['stripe.reporting_category'] == 'issuing_authorization_release']

# get all rows where stripe.source_id is in both dataframes so the two balance out
df_stripe_issuing_authorization_ids = pd.merge(df_stripe_issuing_authorization_hold, df_stripe_issuing_authorization_release, on='stripe.source_id', how='inner')
df_stripe_issuing_authorization_ids.drop_duplicates(subset=['stripe.balance_transaction_id_x'], inplace=True)

df_stripe_authorization = df_stripe_issuing[df_stripe_issuing['stripe.source_id'].isin(df_stripe_issuing_authorization_ids['stripe.source_id'])]

# filter out all rows where stripe.source_id is in df_stripe_issuing_authorization_ids
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_authorization['stripe.balance_transaction_id'])]

# confirm that the sum of the net column of all rows in df_stripe_issuing_authorization_hold is 0, round to avoid floating point errors
print(f'Filtered out {len(df_stripe_authorization)} hold & release transactions')
print(f'Net sum of filtered out hold & release transactions: {round(df_stripe_authorization["stripe.net"].sum(),3)}')
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Filtered out 4586 hold & release transactions
Net sum of filtered out hold & release transactions: 0.0
Unreconciled Stripe transactions: 3062 out of 139879


#### Reconcile completed virtual card transactions

NOTE: For some reason there are virtual card transactions that do not have an expense.virtualCard.id

In [110]:
# Get stripe transactions where stripe.reporting_category is issuing_transaction
df_stripe_issuing_transaction = df_left_stripe[df_left_stripe['stripe.reporting_category'] == 'issuing_transaction']

In [111]:
# Merge the two dataframes on merchantId and source_id
df_stripe_platform_virtual_card = pd.merge(df_platform.dropna(subset=['merchantId']), df_stripe_issuing_transaction.dropna(subset=['stripe.source_id']), left_on='merchantId', right_on='stripe.source_id', how='inner')
df_stripe_platform_virtual_card.drop_duplicates(subset=['stripe.balance_transaction_id'], inplace=True)
print(f'Successfully matched {len(df_stripe_platform_virtual_card)} Stripe transactions with virtual card transactions on platform')

Successfully matched 2180 Stripe transactions with virtual card transactions on platform


In [112]:
# filter out all rows from df_left_stripe where stripe.balance_transaction_id is in df_stripe_platform_virtual_card
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_platform_virtual_card['stripe.balance_transaction_id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Unreconciled Stripe transactions: 882 out of 139879


### Reconcile disputes and dispute reversals

In [113]:
# Find disputes in Stripe by filtering out all rows where stripe.reporting_category is dispute or dispute_reversal
df_stripe_disputes = df_left_stripe[df_left_stripe['stripe.reporting_category'].isin(['dispute', 'dispute_reversal'])]

# Merge platform transactions with disputes on stripe.charge_id
df_stripe_platform_disputes = pd.merge(df_platform.dropna(subset=['merchantId']) , df_stripe_disputes.dropna(subset=['stripe.charge_id']), left_on='merchantId', right_on='stripe.charge_id', how='inner')
df_stripe_platform_disputes.drop_duplicates(subset=['stripe.balance_transaction_id'], inplace=True)

print(f'Successfully matched {len(df_stripe_platform_disputes)} Stripe transactions with disputes on platform')

Successfully matched 63 Stripe transactions with disputes on platform


In [114]:
# filter out all rows from df_left_stripe where stripe.balance_transaction_id is in df_stripe_platform_disputes
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_platform_disputes['stripe.balance_transaction_id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Unreconciled Stripe transactions: 819 out of 139879


### Reconcile virtual card topups

In [115]:
df_stripe_topups = df_left_stripe[df_left_stripe['stripe.reporting_category'] == 'topup']
df_stripe_topup_reversals = df_left_stripe[df_left_stripe['stripe.reporting_category'] == 'topup_reversal']

# find bank transactions that match the topups
df_stripe_bank_topups = matchByDateAndValue(df_stripe_topups, ['stripe.created_utc', 'stripe.net'], df_banks, ['bank.Post Date', 'bank.Debit'], '2 days', 0.05).dropna(subset=['df2key'])
df_stripe_bank_topup_reversals = matchByDateAndValue(df_stripe_topup_reversals, ['stripe.created_utc', 'stripe.net'], df_banks, ['bank.Post Date', 'bank.Credit'], '2 days', 0.05, absolute=True).dropna(subset=['df2key'])

print(f'Successfully matched {len(df_stripe_bank_topups) + len(df_stripe_bank_topup_reversals)} Stripe transactions with bank topups')

Successfully matched 9 Stripe transactions with bank topups


In [116]:
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_bank_topups['stripe.balance_transaction_id'])]
df_left_stripe = df_left_stripe[~df_left_stripe['stripe.balance_transaction_id'].isin(df_stripe_bank_topup_reversals['stripe.balance_transaction_id'])]
print(f'Unreconciled Stripe transactions: {len(df_left_stripe)} out of {len(df_stripe)}')

Unreconciled Stripe transactions: 810 out of 139879


### Analyzing Stripe remainder

In [117]:
# group df_left_stripe by stripe.reporting_category, count and sum gross, fee and net

df_left_stripe.groupby('stripe.reporting_category').agg({'stripe.gross': ['count', 'sum'], 'stripe.fee': ['sum'], 'stripe.net': ['sum']})

stripe.gross           stripe.fee stripe.net
                                  count       sum        sum        sum
stripe.reporting_category                                              
charge                              451   7828.00     635.01    7192.99
charge_failure                       18  -3618.00      43.05   -3661.05
dispute                               3    -75.00      45.00    -120.00
issuing_authorization_hold            1    -36.00       0.00     -36.00
issuing_dispute                       6    290.14       0.00     290.14
issuing_transaction                  42  21188.86       0.00   21188.86
refund                              289   -293.00       0.00    -293.00

### Adding Stripe to reconciliation dict

In [118]:
reconciliation_dict['stripe']['reconciliation']['reconciled_platform']['stripe_platform_contributions'] = df_stripe_platform_contributions
reconciliation_dict['stripe']['reconciliation']['reconciled_platform']['stripe_platform_contrubution_refunds'] = df_stripe_platform_contrubution_refunds
reconciliation_dict['stripe']['reconciliation']['reconciled_platform']['stripe_platform_virtual_card'] = df_stripe_platform_virtual_card
reconciliation_dict['stripe']['reconciliation']['reconciled_platform']['stripe_platform_disputes'] = df_stripe_platform_disputes

reconciliation_dict['stripe']['reconciliation']['reconciled_other']['stripe_application_fees'] = df_stripe_application_fees
reconciliation_dict['stripe']['reconciliation']['reconciled_other']['stripe_authorization'] = df_stripe_authorization

reconciliation_dict['stripe']['reconciliation']['reconciled_bank']['stripe_bank_topups'] = df_stripe_bank_topups
reconciliation_dict['stripe']['reconciliation']['reconciled_bank']['stripe_bank_topup_reversals'] = df_stripe_bank_topup_reversals

reconciliation_dict['stripe']['reconciliation']['unreconciled']['unreconciled_stripe'] = df_left_stripe

In [119]:
assert_reconcilation('stripe', df_stripe, reconciliation_dict)

All stripe transactions accounted for in reconciliation_dict


0

## Paypal reconciliation

We start by looking into Paypal transactions.

Paypal is more complicated than Stripe and Wise, especially when it comes to payouts in other currencies.

In [120]:
# group df_left_stripe by reporting_category, count and sum gross, fee and net

df_paypal.groupby(['paypal.Type', 'paypal.Status', 'paypal.Currency']).agg({'paypal.Gross': ['count', 'sum'], 'paypal.Fee': ['sum'], 'paypal.Net': ['sum']})

paypal.Gross  \
                                                                                 count   
paypal.Type                                 paypal.Status paypal.Currency                
Bank Deposit to PP Account                  Completed     USD                        1   
Cancellation of Hold for Dispute Resolution Denied        USD                       55   
Chargeback                                  Completed     USD                       29   
Chargeback Reversal                         Completed     USD                        5   
Express Checkout Payment                    Completed     BRL                        1   
                                                          EUR                       82   
                                                          GBP                        9   
                                                          USD                     8439   
General Currency Conversion                 Completed     AUD                       21   
                                                          BRL                        2   
                                                          CAD                        9   
                                                          DKK                        1   
                                                          EUR                       71   
                                                          GBP                       21   
                                                          NOK                        2   
                                                          NZD                        3   
                                                          PLN                        1   
                                                          SEK                       15   
                                                          USD                      146   
General Payment                             Completed     BRL                        1   
                                                          USD                       13   
General Withdrawal                          Completed     USD                        5   
Hold on Available Balance                   Completed     EUR                       56   
                                                          GBP                        2   
                                                          USD                     1165   
Hold on Balance for Dispute Investigation   Pending       USD                       53   
Mass Pay Payment                            Completed     AUD                       21   
                                                          CAD                        9   
                                                          DKK                        1   
                                                          EUR                      128   
                                                          GBP                       23   
                                                          NOK                        2   
                                                          NZD                        3   
                                                          PLN                        1   
                                                          SEK                       15   
                                                          USD                     1362   
                                            Pending       EUR                        1   
                                                          USD                       39   
                                            Unclaimed     EUR                        3   
                                                          USD                       19   
Mass Pay Reversal                           Completed     EUR                        1   
                                                          USD                        8   
Mobile Payment          

###  Reconciling holds and hold reversals

In [121]:
hold_types = ['Reversal of General Account Hold', 'Hold on Available Balance', 'Hold on Balance for Dispute Investigation', 'Cancellation of Hold for Dispute Resolution	']
df_paypal_holds = df_paypal[df_paypal['paypal.Type'].isin(hold_types)]
df_left_paypal = df_paypal[~df_paypal['paypal.Transaction ID'].isin(df_paypal_holds['paypal.Transaction ID'])]

print(f'Filtered out {len(df_paypal_holds)} Paypal transactions that are currency conversions or holds')

print(f'Unreconciled Paypal transactions: {len(df_left_paypal)} out of {len(df_paypal)}')

Filtered out 2931 Paypal transactions that are currency conversions or holds
Unreconciled Paypal transactions: 60447 out of 63431


### Reconciling currency conversions

When you make a payout from PayPal in another currency than the currency of the PayPal account, it results in three transactions:
* A debit from PayPal of the amount in USD
* A credit to PayPal of the amount in the target currency
* A payment to the recipient in the target currency

Essentially, what happens is that you sell USD to PayPal from your USD account, then PayPal transfers you EUR into your EUR account, and finally you use that to pay the recipient. 

We do not want to do our accounting in multiple currencies, so we will ignore the currency conversion transactions and only concider the transaction in USD. We must however make a note of that OSC holds small amounts of foreign currency in its Paypal acccount becuase sometimes the payouts bounce and then they get refunded in the target currency. We leave it up to the accountants to choose what to do with these small balances of foreign currency.

We filter out transactions from the Paypal export that relate to internal currency exchange.

In [122]:
conversion_types = ['General Currency Conversion', 'User Initiated Currency Conversion']
df_paypal_currency_conversions = df_left_paypal[df_left_paypal['paypal.Type'].isin(conversion_types)]
df_left_paypal = df_left_paypal[~df_left_paypal['paypal.Transaction ID'].isin(df_paypal_currency_conversions['paypal.Transaction ID'])]

print(f'Filtered out {len(df_paypal_currency_conversions)} Paypal transactions that are currency conversions or holds')

print(f'Unreconciled Paypal transactions: {len(df_left_paypal)} out of {len(df_paypal)}')

Filtered out 308 Paypal transactions that are currency conversions or holds
Unreconciled Paypal transactions: 60139 out of 63431


### Reconciling Paypal contributions

In [123]:
df_platform_contributions = df_platform[df_platform['kind'] == 'CONTRIBUTION']

In [124]:
df_paypal_platform_contributions = pd.merge(df_platform_contributions.dropna(subset=['merchantId']), df_left_paypal.dropna(subset=['paypal.Transaction ID']), left_on='merchantId', right_on='paypal.Transaction ID', how='inner')
df_paypal_platform_contributions.drop_duplicates(subset=['paypal.Transaction ID'], inplace=True)
print(f'Successfully matched {len(df_paypal_platform_contributions)} rows out of {len(df_left_paypal)} Paypal transactions with a platform transaction')
df_left_paypal = df_left_paypal[~df_left_paypal['paypal.Transaction ID'].isin(df_paypal_platform_contributions['paypal.Transaction ID'])]
print(f'Unreconciled Paypal transactions: {len(df_left_paypal)} out of {len(df_paypal)}')

Successfully matched 58315 rows out of 60139 Paypal transactions with a platform transaction
Unreconciled Paypal transactions: 1824 out of 63431


### Reconciling Paypal expenses

In [125]:
df_paypal_platform_expenses = pd.merge(df_platform.dropna(subset=['merchantId']), df_left_paypal.dropna(subset=['paypal.Transaction ID']), left_on='merchantId', right_on='paypal.Transaction ID', how='inner')
df_paypal_platform_expenses.drop_duplicates(subset=['paypal.Transaction ID'], inplace=True)
print(f'Successfully matched {len(df_paypal_platform_expenses)} rows out of {len(df_left_paypal)} Paypal transactions with a platform transaction')
df_left_paypal = df_left_paypal[~df_left_paypal['paypal.Transaction ID'].isin(df_paypal_platform_expenses['paypal.Transaction ID'])]
print(f'Unreconciled Paypal transactions: {len(df_left_paypal)} out of {len(df_paypal)}')

Successfully matched 1607 rows out of 1824 Paypal transactions with a platform transaction
Unreconciled Paypal transactions: 217 out of 63431


### Reconcile Paypal bank topups and withdrawals 

In [126]:
df_paypal_withdrawals = df_left_paypal[df_left_paypal['paypal.Type'] == 'General Withdrawal']
df_paypal_deposits = df_left_paypal[df_left_paypal['paypal.Type'] == 'Bank Deposit to PP Account']

# find bank transactions that match the topups
df_paypal_bank_withdrawals = matchByDateAndValue(df_paypal_withdrawals, ['paypal.Datetime', 'paypal.Net'], df_banks, ['bank.Post Date', 'bank.Credit'], '5 days', 0.05, absolute=True).dropna(subset=['df2key'])
df_paypal_bank_deposits = matchByDateAndValue(df_paypal_deposits, ['paypal.Datetime', 'paypal.Net'], df_banks, ['bank.Post Date', 'bank.Debit'], '5 days', 0.05, absolute=True).dropna(subset=['df2key'])

print(f'Successfully matched {len(df_paypal_bank_withdrawals) + len(df_paypal_bank_deposits)} Paypal transactions with bank topups')

df_left_paypal = df_left_paypal[~df_left_paypal['paypal.Transaction ID'].isin(df_paypal_bank_withdrawals['paypal.Transaction ID'])]
df_left_paypal = df_left_paypal[~df_left_paypal['paypal.Transaction ID'].isin(df_paypal_bank_deposits['paypal.Transaction ID'])]
print(f'Unreconciled Paypal transactions: {len(df_left_paypal)} out of {len(df_paypal)}')


Successfully matched 6 Paypal transactions with bank topups
Unreconciled Paypal transactions: 211 out of 63431


### Analyzing Paypal remainder

In [127]:
# group df_left_stripe by reporting_category, count and sum gross, fee and net

df_left_paypal.groupby(['paypal.Type', 'paypal.Status', 'paypal.Currency']).agg({'paypal.Gross': ['count', 'sum'], 'paypal.Fee': ['sum'], 'paypal.Net': ['sum']})

paypal.Gross  \
                                                                                 count   
paypal.Type                                 paypal.Status paypal.Currency                
Cancellation of Hold for Dispute Resolution Denied        USD                        2   
Chargeback                                  Completed     USD                       29   
Chargeback Reversal                         Completed     USD                        5   
Express Checkout Payment                    Completed     EUR                        1   
                                                          USD                       29   
General Payment                             Completed     BRL                        1   
                                                          USD                       13   
Mass Pay Payment                            Completed     EUR                        2   
                                                          USD                        9   
                                            Pending       USD                        1   
                                            Unclaimed     EUR                        1   
                                                          USD                        7   
Mass Pay Reversal                           Completed     EUR                        1   
                                                          USD                        8   
Mobile Payment                              Completed     USD                        3   
Payment Refund                              Completed     USD                       46   
Payment Reversal                            Completed     USD                        1   
Subscription Payment                        Completed     USD                       52   

                                                                                    \
                                                                               sum   
paypal.Type                                 paypal.Status paypal.Currency            
Cancellation of Hold for Dispute Resolution Denied        USD                18.32   
Chargeback                                  Completed     USD              -350.05   
Chargeback Reversal                         Completed     USD                22.00   
Express Checkout Payment                    Completed     EUR                50.00   
                                                          USD              2329.00   
General Payment                             Completed     BRL              -586.98   
                                                          USD              -795.43   
Mass Pay Payment                            Completed     EUR                28.01   
                                                          USD                49.06   
                                            Pending       USD               -19.80   
                                            Unclaimed     EUR                -4.55   
                                                          USD             -2609.06   
Mass Pay Reversal                           Completed     EUR                 4.55   
                                                          USD              2101.86   
Mobile Payment                              Completed     USD                 1.08   
Payment Refund                              Completed     USD             -3101.99   
Payment Reversal                            Completed     USD                -5.00   
Subscription Payment                        Completed     USD              3224.55   

                                                                          paypal.Fee  \
                                                                                 sum   
paypal.Type                                 paypal.Status paypal.Currency              
Cancellation of Hold for Dispute Resolution Denied        USD                   0.00   
Chargeback              

### Adding Paypal to reconciliation dict

TODO: Fix paypal transaction discrepancy 
TODO: Separate Paypal df_paypal_platform by contribution, expense, fees, etc

In [128]:
reconciliation_dict['paypal']['reconciliation']['reconciled_platform']['paypal_platform_contributions'] = df_paypal_platform_contributions
reconciliation_dict['paypal']['reconciliation']['reconciled_platform']['paypal_platform_expenses'] = df_paypal_platform_expenses

reconciliation_dict['paypal']['reconciliation']['reconciled_bank']['paypal_bank_withdrawals'] = df_paypal_bank_withdrawals
reconciliation_dict['paypal']['reconciliation']['reconciled_bank']['paypal_bank_deposits'] = df_paypal_bank_deposits

reconciliation_dict['paypal']['reconciliation']['reconciled_other']['paypal_holds'] = df_paypal_holds
reconciliation_dict['paypal']['reconciliation']['reconciled_other']['paypal_currency_conversions'] = df_paypal_currency_conversions

reconciliation_dict['paypal']['reconciliation']['unreconciled']['unreconciled_paypal'] = df_left_paypal

In [129]:
if assert_reconcilation('paypal', df_paypal, reconciliation_dict) != 0:
    df_paypal_drop_duplicates = df_paypal.drop_duplicates(subset=['paypal.Transaction ID'], inplace=False)
    # Check so all transactions in df_paypal - df_paypal_drop_duplicates have Type in hold_types
    duplicates_are_hold = len(df_paypal[~df_paypal['paypal.Transaction ID'].isin(df_paypal_drop_duplicates['paypal.Transaction ID'])][~df_paypal[~df_paypal['paypal.Transaction ID'].isin(df_paypal_drop_duplicates['paypal.Transaction ID'])]['paypal.Type'].isin(hold_types)]) == 0
    if duplicates_are_hold:
        print('All duplicates are hold transactions. After removing hold duplicates:')
        assert_reconcilation('paypal', df_paypal_drop_duplicates, reconciliation_dict)

53 paypal transactions not accounted for in reconciliation_dict
All duplicates are hold transactions. After removing hold duplicates:
All paypal transactions accounted for in reconciliation_dict


## Internal host transfer reconciliation

We filter out the internal transfers between collectives and between the host and collectives and funds and collectives.
We put these transactions aside for later, they will be used for accounting.

In [130]:
# create a df_internal_host_transactions dataframe that contains all platform transactions with paymentMethod.service == 'OPENCOLLECTIVE'
df_internal_host_transactions = df_platform[df_platform['paymentMethod.service'] == 'OPENCOLLECTIVE']

## Bank reconciliation

### Filter out all platform transactions that could be bank transactions

We now want to reconcile bank account transactions with platform transactions.

Identify platform transactions that might be bank related as follows:
* Filter out all transaction kinds except contributions, expenses and added funds
* Filter out known Wise, Paypal and Stripe transactions
* Filter out platform transactions between collectives and from funds

In [131]:
df_platform_bank_related = df_platform[(df_platform['kind'] == 'CONTRIBUTION') | (df_platform['kind'] == 'EXPENSE') | (df_platform['kind'] == 'ADDED_FUNDS')]

# filter out all platform transactions that are already reconciled with Wise, Stripe or Paypal
for key, value in reconciliation_dict.items():
    for key2, value2 in value['reconciliation']['reconciled_platform'].items():
        df_platform_bank_related = df_platform_bank_related[~df_platform_bank_related['id'].isin(value2['id'])]

# filter out transactions in df_platform_bank_related where paymentMethod.service == 'OPENCOLLECTIVE', these are internal transactions
df_platform_bank_related = df_platform_bank_related[~df_platform_bank_related['id'].isin(df_internal_host_transactions['id'])]

### Mark reconciled bank transactions from Wise, Stripe and Paypal reconciliation 

We have already reconciled some transactions with the bank and Wise, Stripe and Paypal

In [132]:
df_bank_stripe_topups = pd.concat([value for key, value in reconciliation_dict['stripe']['reconciliation']['reconciled_bank'].items()])
df_bank_wise = pd.concat([value for key, value in reconciliation_dict['wise']['reconciliation']['reconciled_bank'].items()])
df_bank_paypal = pd.concat([value for key, value in reconciliation_dict['paypal']['reconciliation']['reconciled_bank'].items()])

df_left_bank = df_banks[~df_banks['bank.accountindex'].isin(df_bank_stripe_topups['bank.accountindex'])]
df_left_bank = df_left_bank[~df_left_bank['bank.accountindex'].isin(df_bank_wise['bank.accountindex'])]
df_left_bank = df_left_bank[~df_left_bank['bank.accountindex'].isin(df_bank_paypal['bank.accountindex'])]

print(f'Unreconciled bank transactions: {len(df_left_bank)} out of {len(df_banks)}')

Unreconciled bank transactions: 611 out of 660


### Idenfify Stripe payouts

In [133]:
# find bank transactions with detail that contains 'STRIPE TRANSFER', trimming whitespace before matching
df_bank_stripe_transfers = df_left_bank[df_left_bank['bank.Detail'].str.replace(r'\s+', '', regex=True).str.contains('STRIPETRANSFER')]

print(f'Filtered out {len(df_bank_stripe_transfers)} bank transactions that are Stripe transfers')

df_left_bank = df_left_bank[~df_left_bank['bank.accountindex'].isin(df_bank_stripe_transfers['bank.accountindex'])]
print(f'Unreconciled bank transactions: {len(df_left_bank)} out of {len(df_banks)}')

Filtered out 19 bank transactions that are Stripe transfers
Unreconciled bank transactions: 592 out of 660


### Match transactions on date, amount and string comparison

Match transactions on bank and platform that happen within a 10 day window, where the values are within 2.5%, and there is a high scoring substring match between the Detail column of the bank transaction and the oppositeAccount.name of the platform transaction.

In [134]:
df_matches = matchByDateAmountString(df_left_bank, df_platform_bank_related, s1='bank.Detail', s2='oppositeAccount.name', min_similarity=80, max_days=10, max_diff=0.025)
# Merge the matches dataframe with df_left_bank
df_bank_matched = pd.merge(df_left_bank, df_matches, left_on='bank.accountindex', right_on='bank_accountindex', how='inner')
# Merge the df_bank_platform_matched with df_platform_bank_related
df_bank_platform_matched = pd.merge(df_platform_bank_related, df_bank_matched, right_on='platform_id',  left_on='id', how='inner')
# Drop the temporary columns used for merging
df_bank_platform_matched.drop(columns=['bank_accountindex', 'platform_id'], inplace=True)
df_bank_platform_matched.drop_duplicates(subset=['bank.accountindex'], inplace=True)
df_left_bank = df_left_bank[~df_left_bank['bank.accountindex'].isin(df_bank_platform_matched['bank.accountindex'])]
print(f'In first run, matched {len(df_bank_platform_matched)} bank transactions with platform transactions')

# Run matching again on remainder, but this time decrease the time window. 
# Because the matchByDateAmountString only returns a match if it finds exactly one match, we should find more in the second run.
# We decrease the time window to avoid ambiguous matches.
df_left_platform_bank_related = df_platform_bank_related[~df_platform_bank_related['id'].isin(df_bank_platform_matched['id'])]
df_matches_2 = matchByDateAmountString(df_left_bank, df_left_platform_bank_related, s1='bank.Detail', s2='oppositeAccount.name', min_similarity=80, max_days=10, max_diff=0.025)
df_bank_matched_2 = pd.merge(df_left_bank, df_matches_2, left_on='bank.accountindex', right_on='bank_accountindex', how='inner')
df_bank_platform_matched_2 = pd.merge(df_left_platform_bank_related, df_bank_matched_2, right_on='platform_id',  left_on='id', how='inner')
df_bank_platform_matched_2.drop(columns=['bank_accountindex', 'platform_id'], inplace=True)
df_bank_platform_matched_2.drop_duplicates(subset=['bank.accountindex'], inplace=True)
df_left_bank = df_left_bank[~df_left_bank['bank.accountindex'].isin(df_bank_platform_matched_2['bank.accountindex'])]
print(f'In second run, matched {len(df_bank_platform_matched_2)} bank transactions with platform transactions')

# concatenate the two dataframes
df_bank_platform_matched = pd.concat([df_bank_platform_matched, df_bank_platform_matched_2])
df_bank_platform_matched.drop_duplicates(subset=['bank.accountindex'], inplace=True)

print(f'Successfully matched {len(df_bank_platform_matched)} bank transactions with platform transactions')
print(f'Unreconciled bank transactions: {len(df_left_bank)} out of {len(df_banks)}')

In first run, matched 164 bank transactions with platform transactions
In second run, matched 3 bank transactions with platform transactions
Successfully matched 167 bank transactions with platform transactions
Unreconciled bank transactions: 425 out of 660


### Adding bank to reconciliation dict

In [135]:
reconciliation_dict['bank']['reconciliation']['reconciled_wise']['bank_wise'] = df_bank_wise
reconciliation_dict['bank']['reconciliation']['reconciled_stripe']['bank_stripe_topups'] = df_bank_stripe_topups
reconciliation_dict['bank']['reconciliation']['reconciled_stripe']['bank_stripe_transfers'] = df_bank_stripe_transfers
reconciliation_dict['bank']['reconciliation']['reconciled_paypal']['bank_paypal'] = df_bank_paypal

reconciliation_dict['bank']['reconciliation']['reconciled_platform']['bank_platform_matched'] = df_bank_platform_matched
reconciliation_dict['bank']['reconciliation']['unreconciled']['unreconciled_bank'] = df_left_bank

In [136]:
assert_reconcilation('bank', df_banks, reconciliation_dict)

All bank transactions accounted for in reconciliation_dict


0

## Platform ledger validation

In [137]:
# filtering out reconciled transactions from df_left_bank
df_left_platform = df_platform
for key, value in reconciliation_dict.items():
    for key2, value2 in value['reconciliation']['reconciled_platform'].items():
        df_left_platform = df_left_platform[~df_left_platform['id'].isin(value2['id'])]

In [138]:
# filtering out internal transactions from df_left_platform
df_left_platform = df_left_platform[~df_left_platform['id'].isin(df_internal_host_transactions['id'])]

In [139]:
# filter out host fees and host fee share and host fee share debt transactions from df_left_platform
df_left_platform = df_left_platform[~df_left_platform['kind'].isin(['HOST_FEE', 'HOST_FEE_SHARE', 'HOST_FEE_SHARE_DEBT'])]

In [140]:
df_left_platform.groupby('kind').agg({'type': ['count']})

,type
,count
kind,
BALANCE_TRANSFER,4
CONTRIBUTION,699
EXPENSE,305
PAYMENT_PROCESSOR_COVER,728
PAYMENT_PROCESSOR_DISPUTE_FEE,18
PLATFORM_TIP_DEBT,11
PREPAID_PAYMENT_METHOD,13


In [141]:
print(f'Unreconciled platform transactions: {len(df_left_platform)} out of {len(df_platform)}')

Unreconciled platform transactions: 1778 out of 845230


# Total remainder

In [142]:
# Create a dataframe with the unreconciled transaction categories and counts

df_unreconciled_counts = pd.DataFrame([
    ['Stripe', len(df_left_stripe)],
    ['Wise', len(df_left_wise)],
    ['Paypal', len(df_left_paypal)],
    ['Bank', len(df_left_bank)]
], columns=['type', 'unreconciled_count'])

df_unreconciled_counts

,type,unreconciled_count
0,Stripe,810
1,Wise,26
2,Paypal,211
3,Bank,425


In [143]:
print(f'Sum of unreconciled counts: {df_unreconciled_counts["unreconciled_count"].sum()}')

Sum of unreconciled counts: 1472


# Report

In [144]:
# Print a summary of the reconciliation
print('Reconciliation summary')
print('')

print(f'Wise transactions: {len(df_wise)}')
print(f'Wise transactions reconciled with platform: {len(df_wise_platform)}')
print(f'Wise transactions reconciled with bank: {len(df_wise_bank)}')
print(f'Wise transactions unreconciled: {len(df_left_wise)}')

print('')

print(f'Stripe transactions: {len(df_stripe)}')
print(f'Stripe transactions reconciled with platform: {len(df_stripe_platform_contributions) + len(df_stripe_platform_contrubution_refunds) + len(df_stripe_platform_virtual_card) + len(df_stripe_platform_disputes)}')
print(f'Stripe transactions reconciled with bank: {len(df_stripe_bank_topups) + len(df_stripe_bank_topup_reversals)}')
print(f'Stripe transactions unreconciled: {len(df_left_stripe)}')

print('')

print(f'Paypal transactions: {len(df_paypal)}')
print(f'Paypal transactions reconciled with platform: {len(df_paypal_platform_contributions) + len(df_paypal_platform_expenses)}')
print(f'Paypal transactions reconciled with bank: {len(df_paypal_bank_withdrawals) + len(df_paypal_bank_deposits)}')
print(f'Paypal transactions unreconciled: {len(df_left_paypal)}')

print('')

print(f'Bank transactions: {len(df_banks)}')
print(f'Bank transactions reconciled with stripe: {len(df_bank_stripe_topups) + len(df_bank_stripe_transfers)}')
print(f'Bank transactions reconciled with wise: {len(df_bank_wise)}')
print(f'Bank transactions reconciled with paypal: {len(df_bank_paypal)}')
print(f'Bank transactions with reconciliation suggestions with platform: {len(df_bank_platform_matched)}')
print(f'Bank transactions unreconciled: {len(df_left_bank)}')

print('')

print(f'Total transactions on Wise, Stripe, Paypal and Banks: {len(df_wise) + len(df_stripe) + len(df_paypal) + len(df_banks)}')
print(f'Total transactions reconciled: {len(df_wise_platform) + len(df_wise_bank) + len(df_stripe_platform_contributions) + len(df_stripe_platform_contrubution_refunds) + len(df_stripe_platform_virtual_card) + len(df_stripe_platform_disputes) + len(df_stripe_bank_topups) + len(df_stripe_bank_topup_reversals) + len(df_paypal_platform_contributions) + len(df_paypal_platform_expenses) + len(df_paypal_bank_withdrawals) + len(df_paypal_bank_deposits) + len(df_bank_stripe_topups) + len(df_bank_stripe_transfers) + len(df_bank_wise) + len(df_bank_paypal) + len(df_bank_platform_matched)}')
print(f'Total transactions unreconciled: {len(df_left_wise) + len(df_left_stripe) + len(df_left_paypal) + len(df_left_bank)}')
print(f'Percentage unreconciled: {round((len(df_left_wise) + len(df_left_stripe) + len(df_left_paypal) + len(df_left_bank))/(len(df_wise) + len(df_stripe) + len(df_paypal) + len(df_banks))*100, 2)}%')

Reconciliation summary

Wise transactions: 3463
Wise transactions reconciled with platform: 3403
Wise transactions reconciled with bank: 34
Wise transactions unreconciled: 26

Stripe transactions: 139879
Stripe transactions reconciled with platform: 134007
Stripe transactions reconciled with bank: 9
Stripe transactions unreconciled: 810

Paypal transactions: 63431
Paypal transactions reconciled with platform: 59922
Paypal transactions reconciled with bank: 6
Paypal transactions unreconciled: 211

Bank transactions: 660
Bank transactions reconciled with stripe: 28
Bank transactions reconciled with wise: 34
Bank transactions reconciled with paypal: 6
Bank transactions with reconciliation suggestions with platform: 167
Bank transactions unreconciled: 425

Total transactions on Wise, Stripe, Paypal and Banks: 207433
Total transactions reconciled: 197616
Total transactions unreconciled: 1472
Percentage unreconciled: 0.71%


# Export to CSV

In [145]:
reportDir = './reports/opensource'

In [146]:
def process_nested_dict_to_df(nested_dict):
    frames = []
    for key, value in nested_dict.items():
        if isinstance(value, pd.DataFrame):
            if not value.empty:  # Check if the DataFrame is not empty
                # Use .copy() to ensure you're working with a copy of the DataFrame
                temp_df = value.copy()
                temp_df['_reconciliation_category'] = key
                frames.append(temp_df)
        else:
            frames.extend(process_nested_dict_to_df(value))
    return frames


In [147]:
# Process the nested dictionary and concatenate all dataframes into one
frames = process_nested_dict_to_df(reconciliation_dict['wise']['reconciliation'])
wise_export = pd.concat(frames, ignore_index=True)
# reorder the columns so that columns starting with wise are first
wise_export = wise_export[sorted(wise_export.columns, key=lambda x: x.startswith('wise.'), reverse=True)]

# assert that length of wise_export is equal to length of df_wise
assert len(wise_export) == len(df_wise)

In [148]:
# Process the nested dictionary and concatenate all dataframes into one
frames = process_nested_dict_to_df(reconciliation_dict['stripe']['reconciliation'])
stripe_export = pd.concat(frames, ignore_index=True)
# reorder the columns so that columns starting with stripe are first
stripe_export = stripe_export[sorted(stripe_export.columns, key=lambda x: x.startswith('stripe.'), reverse=True)]

# assert that length of stripe_export is equal to length of df_stripe
assert len(stripe_export) == len(df_stripe)

In [149]:
# Process the nested dictionary and concatenate all dataframes into one
frames = process_nested_dict_to_df(reconciliation_dict['paypal']['reconciliation'])
paypal_export = pd.concat(frames, ignore_index=True)
# reorder the columns so that columns starting with paypal are first
paypal_export = paypal_export[sorted(paypal_export.columns, key=lambda x: x.startswith('paypal.'), reverse=True)]

# assert that length of paypal_export is equal to length of df_paypal
assert len(paypal_export) == len(df_paypal_drop_duplicates)

In [150]:
# Process the nested dictionary and concatenate all dataframes into one
frames = process_nested_dict_to_df(reconciliation_dict['bank']['reconciliation'])
bank_export = pd.concat(frames, ignore_index=True)
# reorder the columns so that columns starting with bank are first
bank_export = bank_export[sorted(bank_export.columns, key=lambda x: x.startswith('bank.'), reverse=True)]

# assert that length of bank_export is equal to length of df_banks
assert len(bank_export) == len(df_banks)

In [152]:
# save exports to csv 
wise_export.to_csv(f'{reportDir}/wise_reconciliation.csv', index=False)
stripe_export.to_csv(f'{reportDir}/stripe_reconciliation.csv', index=False)
paypal_export.to_csv(f'{reportDir}/paypal_reconciliation.csv', index=False)
bank_export.to_csv(f'{reportDir}/bank_reconciliation.csv', index=False)
df_platform.to_csv(f'{reportDir}/all_platform_transactions.csv', index=False)

# save unreconciled counts to csv
df_unreconciled_counts.to_csv(f'{reportDir}/unreconciled_counts.csv', index=False)